In [1]:
import datetime
import re, os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
class Config:
    bos_token = '<s>'
    eos_token = '</s>'
    usr_token = '<usr>'
    pad_token = '<pad>'
    sys_token = '<sys>'
    unk_token = '<unk>'
    mask_token = '<mask>'
    max_length = 2 ** 8
    max_turns = 3
    pretrained_model_name = "skt/kogpt2-base-v2"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_name = 'model.pt'

    

In [3]:
tokenizer = AutoTokenizer.from_pretrained(Config.pretrained_model_name,
            bos_token=Config.bos_token, eos_token=Config.eos_token, unk_token=Config.unk_token,
            pad_token=Config.pad_token, mask_token=Config.mask_token, model_max_length=Config.max_length)
model = AutoModelForCausalLM.from_pretrained(Config.pretrained_model_name).to(Config.device)


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [ ]:
model.load_state_dict(torch.load(Config.model_name))

In [ ]:
bad_words_ids = [tokenizer.encode(token) for token in [Config.unk_token,
                'ㅋ', 'ㅎ', 'ㅋㅋ', 'ㅎㅎ', 'ㅜ', 'ㅠ', 'ㅜㅜ', 'ㅠㅠ']]
history_limit = [Config.bos_token]
usr_token_id = tokenizer.convert_tokens_to_ids(Config.usr_token)
sys_token_id = tokenizer.convert_tokens_to_ids(Config.sys_token)

while True:
    user_message = input("user > ")

    if user_message == "끝": break
        
    if user_message == "초기화":
        history_limit = ['<s>']
        continue
        
    if len(history_limit) == Config.max_turns * 2 + 1:
        history_limit = history_limit[: 1] + history_limit[3: ]
        
    user_message_pull = Config.usr_token + user_message + Config.sys_token

    history_limit.append(user_message_pull)

    message_ids = tokenizer.encode(''.join(history_limit),
                                   return_tensors="pt").to(Config.device)

    with torch.no_grad():
        reply_ids = model.generate(
            message_ids,
            max_new_tokens=30,
            early_stopping=True,
            num_beams=3, # ★
            temperature=0.95, # ★
            top_p=0.92, # ★
            repetition_penalty=1.1, # ★
            no_repeat_ngram_size=3,
            bad_words_ids=bad_words_ids,
            forced_eos_token_id=usr_token_id,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=[usr_token_id, sys_token_id]
        )
        
    decoded_ids = reply_ids[0, message_ids.shape[-1]: -1]

    decoded_message = tokenizer.decode(decoded_ids)
    
    history_limit.append(decoded_message)

    print("system > ", decoded_message)
    

In [ ]:
bad_words_ids = [tokenizer.encode(token) for token in [Config.unk_token,
                'ㅋ', 'ㅎ', 'ㅋㅋ', 'ㅎㅎ', 'ㅜ', 'ㅠ', 'ㅜㅜ', 'ㅠㅠ']]
history_limit = [Config.bos_token]

while True:
    user_message = input("user > ")

    if user_message == "끝": break
        
    if user_message == "초기화":
        history_limit = ['<s>']
        continue
        
    if len(history_limit) == Config.max_turns * 2 + 1:
        history_limit = history_limit[: 1] + history_limit[3: ]
        
    user_message_pull = Config.usr_token + user_message + Config.sys_token

    history_limit.append(user_message_pull)

    message_ids = tokenizer.encode(''.join(history_limit),
                                   return_tensors="pt").to(Config.device)

    with torch.no_grad():
        reply_ids = model.generate(
            message_ids,
            max_new_tokens=30,
            do_sample=True,
            top_k=10,
            temperature=0.95,
            top_p=0.92,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3,
            bad_words_ids=bad_words_ids,
            forced_eos_token_id=usr_token_id,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=[usr_token_id, sys_token_id]
        )
        
    system_ids = reply_ids[0, message_ids.shape[-1]: -1]

    decoded_message = tokenizer.decode(decoded_ids)
    
    history_limit.append(decoded_message)

    print("system > ", decoded_message)
    

In [ ]:
system_ids